In [16]:
import pandas as pd

df = pd.read_csv("homepage_data.csv")
df2 = pd.read_csv("homepage_data_3.csv")
df = df[~df['Personal Website'].str.endswith('.sg').astype(bool)]
df = pd.concat([df, df2], ignore_index=True)
df.head()

,Name,Personal Website,About Us Text
0,Albert Chi-Shing CHUNG,https://cse.hkust.edu.hk/admin/people/faculty/...,Albert Chung is a Full Professor at the Depart...
1,Andrew Wing-On POON,https://eeawpoon.people.ust.hk/,Professor Poon received his BA degree from the...
2,Antoni Bert CHAN,https://www.cs.cityu.edu.hk/~abchan/,Dr. Antoni Chan is a Professor at the City Uni...
3,Brian Kan-Wing MAK,https://cse.hkust.edu.hk/admin/people/faculty/...,Dr. Mak is currently an Associate Professor in...
4,CHAN Chee Yong,https://sgps.cuhk.edu.hk/profile/professor-cha...,Professor Chee Hon Chan is an Assistant Profes...


In [17]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import requests
from markdownify import markdownify as md
import json
from tqdm import tqdm
from prompts import SYSTEM_PROMPT_ANALYZE_HOMEPAGES
from concurrent.futures import ThreadPoolExecutor

load_dotenv(".env")

models = ["openai/gpt-4o-2024-11-20"]
model_tags  = ["gpt-4o"]
iterations = 5

client = OpenAI(
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url="https://openrouter.ai/api/v1",
)

def process_homepage_text(text: str):
    all_responses = {}
    with ThreadPoolExecutor() as executor:
        futures = []
        future_to_info = {}
        for iteration in range(iterations):
            for (model_tag, model_name) in zip(model_tags, models):
                future = executor.submit(
                    client.chat.completions.create,
                    messages=[{
                        "role": "system",
                        "content": SYSTEM_PROMPT_ANALYZE_HOMEPAGES
                    }, {
                        "role": "user",
                        "content": text
                    }],
                    response_format={
                        "type": "json_object"
                    },
                    temperature=0.5,
                    model=model_name
                )
                futures.append(future)
                future_to_info[future] = {
                    "model_tag": model_tag,
                    "model_name": model_name,
                    "iteration": iteration
                }

        for future_idx, future in tqdm(enumerate(futures), desc="Processing"):
            response = future.result()
            run_info = future_to_info[future]
            iteration = run_info["iteration"]
            model_tag = run_info["model_tag"]

            try: 
                response = json.loads(response.choices[0].message.content)
                for key in response:
                    all_responses[f"{key}_{model_tag}_{iteration}"] = response[key]
            except:
                print("Error parsing message, leaving elements blank:", response)
    return all_responses


In [18]:
researcher_profile = []

In [19]:
from rich import print

# TODO: append university profile w/ personal website text
# TODO: make sure that results aren't correlated w/ more text (more personable because they wrote more) 


def process_row(row):
    text = row['About Us Text']
    if not isinstance(text, str):
        return {
            "name": row["Name"],
            "error": "No text found"
        }
    if len(text.strip()) == 0:
        return {
            "name": row["Name"],
            "error": "No text found"
        }
    response = process_homepage_text(text)
    response["name"] = row['Name']
    return response

for index, row in tqdm(list(df.iterrows()), total=df.shape[0]):
    researcher_profile.append(process_row(row))

Processing: 5it [00:17,  3.53s/it]it/s]
Processing: 5it [00:13,  2.73s/it]5, 17.64s/it]
Processing: 5it [00:14,  2.91s/it]0, 15.31s/it]
Processing: 5it [00:12,  2.53s/it]2, 14.98s/it]
Processing: 5it [00:15,  3.11s/it]2, 14.08s/it]
Processing: 5it [00:17,  3.44s/it]4, 14.62s/it]
Processing: 5it [00:15,  3.04s/it]9, 15.50s/it]
Processing: 5it [00:12,  2.59s/it]0, 15.41s/it]
Processing: 5it [00:18,  3.75s/it]1, 11.10s/it]
Processing: 5it [00:24,  4.91s/it]33, 13.08s/it]
Processing: 5it [00:21,  4.25s/it]11, 16.15s/it]
Processing: 5it [00:15,  3.20s/it]31, 17.58s/it]
Processing: 5it [00:19,  3.98s/it]07, 17.13s/it]
Processing: 5it [00:14,  2.83s/it]51, 17.94s/it]
Processing: 5it [00:16,  3.29s/it]51, 16.85s/it]
Processing: 5it [00:16,  3.37s/it]18, 16.75s/it]
Processing: 5it [00:12,  2.49s/it]07, 16.78s/it]
Processing: 5it [00:24,  4.85s/it]42, 15.50s/it]
Processing: 5it [00:16,  3.37s/it]46, 18.11s/it]
Processing: 5it [00:14,  2.89s/it]34, 17.74s/it]
Processing: 5it [00:12,  2.50s/it]57,

Error parsing message, leaving elements blank:
ChatCompletion(
    id='gen-1742233278-MFvf6c7P1ScrzDKLT8QY',
    choices=[
        Choice(
            finish_reason=None,
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content='{\n  "general_analysis": "Dr. Yang Qingxiong\'s webpage provides a detailed overview of 
his academic and professional achievements. It highlights his research interests in computer vision, computer 
graphics, and image processing, as well as his educational background and professional trajectory. The webpage 
emphasizes his awards, publications, and patents, which collectively underscore his expertise and contributions to 
the field. There is little explicit reference to regional or identity-based affiliations, suggesting a focus on 
global academic recognition rather than local identity. The webpage does not prominently feature any cross-group 
projects, unifying language, or outreach initiatives, which could indicate limited emphasis on reconciliation or 
bridge-building in the context of his professional profile. Instead, the focus appears to be on individual 
accomplishments and technical contributions.",\n  "cross_group_thinking_ideas_analysis": "There is no explicit 
mention of cross-group projects or collaborations that span contentious boundaries or regions. The webpage 
primarily focuses on Dr. Yang\'s individual achievements, research contributions, and collaborations with his 
advisors and peers during his academic journey. While his collaborations with researchers from different 
institutions could be considered cross-group in a general sense, there is no indication that these collaborations 
were aimed at reconciliation or bridging divisive lines.",\n  "cross_group_thinking_score": 3,\n  
"unifying_language_ideas_analysis": "The language on the webpage is neutral and professional, focusing on technical
achievements and research contributions. There is no use of unifying or bridge-building language such as 
\'collaborating across diverse backgrounds\' or \'bridging boundaries.\' The absence of such language suggests that
the webpage does not aim to emphasize inclusivity or reconciliation explicitly.",\n  "unifying_language_score": 
2,\n  "shared_initiatives_outreach_ideas_analysis": "The webpage does not highlight any shared initiatives, 
outreach efforts, or projects aimed at fostering collaboration or reconciliation between groups. The focus remains 
on individual research outputs and achievements, with no mention of committee memberships, workshops, or labs that 
bring together diverse groups.",\n  "shared_initiatives_outreach_score": 2,\n  
"importance_of_career_standing_ideas_analysis": "Dr. Yang\'s webpage strongly emphasizes his academic achievements,
awards, and contributions to his field, such as best paper awards, patents,',
                refusal=None,
                role='assistant',
                function_call=None,
                tool_calls=None
            )
        )
    ],
    created=1742233278,
    model='openai/gpt-4o-2024-11-20',
    object='chat.completion',
    service_tier=None,
    system_fingerprint='fp_4dd9e4e381',
    usage=CompletionUsage(completion_tokens=452, prompt_tokens=3266, total_tokens=3718),
    provider='OpenAI'
)

Processing: 5it [00:19,  3.89s/it]
 96%|█████████▋| 158/164 [43:54<02:02, 20.49s/it]

Error parsing message, leaving elements blank:
ChatCompletion(
    id='gen-1742233297-W7UIuYKfzZBjm0m0TyLA',
    choices=[
        Choice(
            finish_reason=None,
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content='{\n  "general_analysis": "Dr. Nutanong\'s webpage provides a professional overview of his 
academic background, research interests, and career trajectory. Notably, the text emphasizes his qualifications and
affiliations with prestigious institutions such as the University of Melbourne, University of Maryland, and Johns 
Hopkins University. There is a focus on his expertise in computer science topics like data-intensive computing, 
scalable machine learning, and scientific data management. However, there is no explicit mention of cross-group 
collaborations, unifying language, or outreach initiatives that bridge diverse identities or regions. The webpage 
is highly focused on his technical expertise and career achievements, with little emphasis on regional or cultural 
identity markers, including his affiliation with Hong Kong.",\n  "cross_group_thinking_ideas_analysis": "The 
webpage does not explicitly highlight cross-group projects or collaborations. It primarily focuses on Dr. 
Nutanong\'s individual research interests and career experiences. While his work spans institutions in Australia, 
the United States, and Hong Kong, there is no mention of projects that bridge diverse groups or regions. The 
absence of such references suggests that cross-group thinking is not a prominent theme in the presentation of his 
profile.",\n  "cross_group_thinking_score": "3",\n  "unifying_language_ideas_analysis": "The language used in the 
profile is neutral and professional, focusing on technical expertise and career milestones. There is no overt use 
of unifying language aimed at bridging diverse groups or fostering inclusivity. The tone is more academic and 
individualistic, emphasizing Dr. Nutanong\'s personal achievements and research interests rather than collaborative
or reconciliatory efforts.",\n  "unifying_language_score": "2",\n  "shared_initiatives_outreach_ideas_analysis": 
"There is no evidence of shared initiatives or outreach',
                refusal=None,
                role='assistant',
                function_call=None,
                tool_calls=None
            )
        )
    ],
    created=1742233297,
    model='openai/gpt-4o-2024-11-20',
    object='chat.completion',
    service_tier=None,
    system_fingerprint='fp_4dd9e4e381',
    usage=CompletionUsage(completion_tokens=352, prompt_tokens=2765, total_tokens=3117),
    provider='OpenAI'
)

Processing: 5it [00:16,  3.35s/it]
Processing: 5it [00:25,  5.05s/it]:36, 19.39s/it]
Processing: 5it [00:17,  3.45s/it]:24, 21.15s/it]
Processing: 5it [00:13,  2.76s/it]:59, 19.99s/it]
Processing: 5it [00:20,  4.10s/it]:36, 18.14s/it]
Processing: 5it [00:17,  3.50s/it]:18, 18.85s/it]
100%|██████████| 164/164 [45:45<00:00, 16.74s/it]


In [20]:
import os
import json

count = 0
while True:
    if os.path.exists(f"homepage_data_analyzed_multiple_{count}.json"):
        count += 1
    else:
        with open(f"homepage_data_analyzed_multiple_{count}.json", "w+") as f:
            json.dump(researcher_profile, f)
        break

In [21]:
# TODO: save to 0721-month-analysis-data, save to new file